In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [235]:
%%bash
cypher-shell -u neo4j -p neo5j "match (mod:ModelCreation)--(samp:Sample)--(mc:MolecularCharacterization)--(p:Platform) 
where p.type = 'mutation'
return mod.sourcePdxId, samp.sourceSampleId, p.name, p.url;" > sampleByPlatform

In [237]:
%%bash
wc -l sampleByPlatform

576 sampleByPlatform


In [252]:
%%bash
cypher-shell -u neo4j -p neo5j "MATCH (p:Patient)--(ps:PatientSnapshot)--(s:Sample)--(mod:ModelCreation) 
with ps, mod, s, p
MATCH (s)--(mc:MolecularCharacterization)--(ma:MarkerAssociation)
WHERE mc.type = 'mutation'
with ps, mod, p, s, mc, ma
MATCH (mc)--(plat:Platform)
OPTIONAL MATCH (s)--(on:OntologyTerm) 
OPTIONAL MATCH (s)--(tt:TumorType) 

return distinct mod.dataSource,mod.sourcePdxId,s.sourceSampleId,on.label,tt.name, ma.molecularDataString, plat.name, plat.url;" > patientSamplesMut 2>&1

In [245]:
%%bash
wc -l patientSamplesMut

208 patientSamplesMut


In [253]:
%%bash
cypher-shell -u neo4j -p neo5j "MATCH (mod:ModelCreation) return distinct(mod.sourcePdxId);" > modelList

In [254]:
%%bash
wc -l modelList
sed -i -e 's/"/'"'"'/g' -e "/mod.sourcePdxId/d" modelList

4030 modelList


In [256]:
os.chdir('../')

In [262]:
%%bash
#Cypher query to export xenograft information for samples in cypher-shell
while read -r line;
do
echo "$line";
cypher-shell -u neo4j -p neo5j "MATCH (p:Patient)--(ps:PatientSnapshot)--(pss:Sample)--(mod:ModelCreation) 
WHERE mod.sourcePdxId = $line
WITH p,ps,pss,mod
OPTIONAL MATCH (mod)--(sp:Specimen)--(s:Sample)-[:CHARACTERIZED_BY]-(mc:MolecularCharacterization)-[:ASSOCIATED_WITH]-(ma:MarkerAssociation)
WHERE mc.type = 'mutation' 
WITH p, ps, pss, mod, s, mc, ma
OPTIONAL MATCH (mc)--(plat:Platform)
OPTIONAL MATCH (pss)--(on:OntologyTerm) 
OPTIONAL MATCH (pss)--(tt:TumorType)  
return distinct mod.dataSource, mod.sourcePdxId,s.sourceSampleId, on.label, tt.name,plat.name, plat.url, ma.molecularDataString ;" >> "./xenograftSamplesByMod/$line.dense" 2>&1;
done < modelList



'CRL-3084'
'CRL-449'
'CRL-4054'
'CRL-3087'
'CRL-VAN-G'
'CRL-680'
'CRL-430'
'CRL-672'
'CRL-550'
'CRL-792'
'CRL-678'
'CRL-677'
'CRL-676'
'CRL-439'
'CRL-3079'
'CRL-3069'
'CRL-2098'
'CRL-2096'
'CRL-2095'
'CRL-2094'
'CRL-2090'
'CRL-274'
'CRL-393'
'CRL-386'
'CRL-3063'
'CRL-4031'
'CRL-3067'
'CRL-4035'
'CRL-2085'
'CRL-2084'
'CRL-2083'
'CRL-2082'
'CRL-2081'
'CRL-260'
'CRL-498'
'CRL-MRI-H-1579'
'CRL-3052'
'CRL-4142'
'CRL-3054'
'CRL-4023'
'CRL-SMTCA17'
'CRL-3057'
'CRL-SMTCA18'
'CRL-2198'
'CRL-2076'
'CRL-2197'
'CRL-2196'
'CRL-2073'
'CRL-2194'
'CRL-2070'
'CRL-2190'
'CRL-251'
'CRL-LI-011'
'CRL-486'
'CRL-243'
'CRL-488'
'CRL-3040'
'CRL-SMTCA23'
'CRL-SMTCA24'
'CRL-3044'
'CRL-2067'
'CRL-3036'
'CRL-4004'
'CRL-2066'
'CRL-3037'
'CRL-2065'
'CRL-3038'
'CRL-1096'
'CRL-3039'
'CRL-2064'
'CRL-4128'
'CRL-2184'
'CRL-2183'
'CRL-4009'
'CRL-2061'
'CRL-597'
'CRL-233'
'CRL-117'
'CRL-2069'
'CRL-2068'
'CRL-4003'
'CRL-2056'
'CRL-4115'
'CRL-1086'
'CRL-3027'
'CRL-2175'
'CRL-2054'
'CRL-2053'
'CRL-2174'
'CRL-3029'
'CRL-2052'


In [259]:
%%bash 
ls ./xenograftSamplesByMod | wc -l -

33 -


In [ ]:
import os
import pandas as ps
import re

In [ ]:
os.chdir("/home/afollette/jupyter_notebooks/mutationExport/xenograftSamplesByMod/")


In [188]:
sampleByPlatform = ps.read_csv("sampleByPlatform", sep=',', error_bad_lines=False)

b'Skipping line 7632: expected 4 fields, saw 5\nSkipping line 7636: expected 4 fields, saw 5\nSkipping line 7672: expected 4 fields, saw 5\n'


In [193]:
sampleByPlatform.columns

Index(['mod.sourcePdxId', ' samp.sourceSampleId', ' p.name', ' p.url'], dtype='object')

In [214]:
modelDf=ps.read_csv("'CRC0735PR'.dense", sep=',', names=["mod.dataSource", "mod.sourcePdxId", "s.sourceSampleId", "on.label", "tt.name", "ma.molecularDataString"])

In [215]:
modelDf.columns

Index(['mod.dataSource', 'mod.sourcePdxId', 's.sourceSampleId', 'on.label',
       'tt.name', 'ma.molecularDataString'],
      dtype='object')

In [216]:
mutJson = modelDf.iloc[1].at['ma.molecularDataString'].replace("'", "").replace("\\", "").replace("|", ",").replace(" [{", "[{")

In [217]:
mutDf = ps.read_json(mutJson)

In [218]:
mutDf.columns

Index(['biotype', 'codingSequenceChange', 'variantClass', 'codonChange',
       'aminoAcidChange', 'consequence', 'functionalPrediction', 'readDepth',
       'alleleFrequency', 'chromosome', 'seqStartPosition', 'refAllele',
       'altAllele', 'ucscGeneId', 'ncbiGeneId', 'ncbiTranscriptId',
       'ensemblTranscriptId', 'existingVariations', 'genomeAssembly',
       'nucleotideChange', 'marker'],
      dtype='object')

In [219]:
expandedDf.iloc[1].at['sample_id']

' "CRC0735PR"'

In [222]:
expandedDf = ps.DataFrame(columns=['source', 'sample_id', 'cancer_diagnosis', 'cancer_type', 'gene', 'transcript', 'cdna', 'aa', 'consequence', 'platform', 'link_to_protocol'])
expandedDf['source'] = modelDf['mod.dataSource']
expandedDf['sample_id'] = modelDf['s.sourceSampleId']
expandedDf['cancer_diagnosis'] = modelDf['on.label']
expandedDf['cancer_type'] = modelDf['tt.name']
expandedDf['gene'] = mutDf['marker']
expandedDf['transcript'] = mutDf['ensemblTranscriptId']
expandedDf['cdna'] = mutDf['codonChange']
expandedDf['aa'] = mutDf['aminoAcidChange']
expandedDf['consequence'] = mutDf['consequence']
#expandedDf['platform'] = sampleByPlatform.loc[sampleByPlatform[' samp.sourceSampleId'] == expandedDf.iloc[1].at['sample_id'].replace('"', '')]

In [223]:
expandedDf

,source,sample_id,cancer_diagnosis,cancer_type,gene,transcript,cdna,aa,consequence,platform,link_to_protocol
0,mod.dataSource,s.sourceSampleId,on.label,tt.name,KLF5,ENST00000377687,tCa/tTa,S307L,missense_variant,NaN,NaN
1,IRCC-CRC,"""CRC0735PRX0A02003TUMD05000""","""Colorectal Carcinoma""","""Primary""",SMAD3,ENST00000327367,aAa/aGa,K378R,missense_variant,NaN,NaN


In [238]:
sampleByPlatform[sampleByPlatform[' samp.sourceSampleId'].str.contains("CRC0735PRX0A02003TUMD05000")]

,mod.sourcePdxId,samp.sourceSampleId,p.name,p.url
683,CRC0735PR,"""CRC0735PRX0A02003TUMD05000""","""Targeted Next Generation Sequencing""","""/platform/ircc-gene-panel/"""
1575,CRC0735PR,"""CRC0735PRX0A02003TUMD05000""","""TargetedNGS_MUT""","""/platform/ircc-gene-panel/"""


In [225]:
expandedDf.iloc[1].at['sample_id'].replace('"', '').replace(" ", "")

'CRC0735PRX0A02003TUMD05000'

In [231]:
sampleByPlatform[' samp.sourceSampleId']

0                "1302065"
1                "1302065"
2                 "907163"
3                 "907163"
4                "1006909"
               ...        
7781        "T241-primary"
7782        "T241-primary"
7783     "T162-xenograft1"
7784        "T162-primary"
7785        "T162-primary"
Name:  samp.sourceSampleId, Length: 7786, dtype: object